<a href="https://colab.research.google.com/github/copyrightFreeGenAI/copyrightFreeImagesGenAI/blob/main/2.%20Generate%20Images%20with%20Models/Generate_Images_Using_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set Paths
annValInstanceFiles = "/content/gdrive/MyDrive/annotations2014/instances_train2014.json" # Path to benchmark instance json
annValCaptionFiles = "/content/gdrive/MyDrive/annotations2014/captions_train2014.json" # Path to benchmark caption json

source_directory = "/content/gdrive/MyDrive/DATA/COCO/Validation/Images" # Path to directory where validaiton MS COCO images were saved
mitsua_target_directory = "/content/gdrive/MyDrive//DATA/Mitsua" # Path to directory where Mitsua images will be saved
sd21_target_directory = "/content/gdrive/MyDrive/DATA/SD2.1" # Path to directory where SD2.1 images will be saved
juggxl_target_directory = "/content/gdrive/MyDrive/DATA/JuggXL" # Path to directory where JuggXL images will be saved
import os
os.environ["OPENAI_API_KEY"] = "" # OpenAI API Key

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from pycocotools.coco import COCO
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
import torch
from PIL import Image
import requests

In [ ]:
coco=COCO(annValInstanceFiles)
coco_caps=COCO(annValCaptionFiles)
cats = coco.loadCats(coco.getCatIds())

# Mitsua

In [ ]:
model_id = "Mitsua/mitsua-diffusion-one"
pipe = StableDiffusionPipeline.from_pretrained(model_id, safety_checker=None)
pipe = pipe.to("cuda")

In [ ]:
for category in os.listdir(source_directory):
    print(category)  # Log current category being processed

    # Create target save directory for this category
    save_dir = os.path.join(mitsua_target_directory, category)
    os.makedirs(save_dir, exist_ok=True)

    # Get COCO image IDs for this category
    catIds = coco.getCatIds(catNms=[category])
    imgIds = coco.getImgIds(catIds=catIds)

    # Extract numeric image IDs from filenames (before the dot)
    filenames = os.listdir(os.path.join(source_directory, category))
    names_before_dot = [int(os.path.splitext(file)[0]) for file in filenames]

    for image_id in names_before_dot:
        img_path = os.path.join(save_dir, f"{image_id}.jpg")

        # Skip if image already generated
        if os.path.exists(img_path):
            continue

        # Load image and associated captions from COCO
        img = coco.loadImgs(imgIds[image_id])[0]
        annIds = coco_caps.getAnnIds(imgIds=img['id'])
        anns = coco_caps.loadAnns(annIds)

        # Combine all captions into a single prompt
        captions_list = [item['caption'].lower() for item in anns]
        prompt = ' '.join(captions_list)

        # Generate image using the prompt and save it
        image = pipe(prompt).images[0]
        image.save(img_path)

#Stable Diffusion 2.1

In [ ]:
model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, safety_checker=None)
pipe = pipe.to("cuda")

In [ ]:
for category in os.listdir(source_directory):
    print(category)
    save_dir = os.path.join(sd21_target_directory, category)
    os.makedirs(save_dir, exist_ok=True)
    catIds = coco.getCatIds(catNms=[category])
    imgIds = coco.getImgIds(catIds=catIds)
    filenames = os.listdir(os.path.join(source_directory, category))
    names_before_dot = [int(os.path.splitext(file)[0]) for file in filenames]
    for image_id in names_before_dot:
        img_path = os.path.join(save_dir, str(image_id) + ".jpg")
        if os.path.exists(img_path):
            continue
        img = coco.loadImgs(imgIds[image_id])[0]
        annIds = coco_caps.getAnnIds(imgIds=img['id'])
        anns = coco_caps.loadAnns(annIds)
        captions_list = [item['caption'].lower() for item in anns]
        prompt = ' '.join(captions_list)
        image = pipe(prompt).images[0]
        image.save(img_path)

# JuggXL

In [ ]:
pipeline = StableDiffusionXLPipeline.from_pretrained(
      "RunDiffusion/Juggernaut-XL-v9", torch_dtype=torch.float16, variant="fp16", use_safetensors=True, safety_checker=None
  ).to("cuda")

In [ ]:
for category in os.listdir(source_directory):
    print(category)
    save_dir = os.path.join(juggxl_target_directory, category)
    os.makedirs(save_dir, exist_ok=True)
    catIds = coco.getCatIds(catNms=[category])
    imgIds = coco.getImgIds(catIds=catIds)
    filenames = os.listdir(os.path.join(source_directory, category))
    names_before_dot = [int(os.path.splitext(file)[0]) for file in filenames]
    for image_id in names_before_dot:
        img_path = os.path.join(save_dir, str(image_id) + ".jpg")
        if os.path.exists(img_path):
            continue
        img = coco.loadImgs(imgIds[image_id])[0]
        annIds = coco_caps.getAnnIds(imgIds=img['id'])
        anns = coco_caps.loadAnns(annIds)
        captions_list = [item['caption'].lower() for item in anns]
        prompt = ' '.join(captions_list)
        image = pipeline(prompt).images[0]
        image.save(img_path)

# Dall-E-2

In [ ]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
for category in os.listdir(source_directory):
  print(category)
  img_target = os.path.join(target_directory, category)
  os.makedirs(img_target, exist_ok=True)
  catIds = coco.getCatIds(catNms=[category])
  imgIds = coco.getImgIds(catIds=catIds)
  filenames = os.listdir(os.path.join(source_directory, category))
  names_before_dot = [int(os.path.splitext(file)[0]) for file in filenames]
  for image_id in names_before_dot:
    img_path = os.path.join(img_target, str(image_id) + ".jpg")
    if os.path.exists(img_path):
      continue
    img = coco.loadImgs(imgIds[image_id])[0]
    annIds = coco_caps.getAnnIds(imgIds=img['id'])
    anns = coco_caps.loadAnns(annIds)
    captions_list = [item['caption'].lower() for item in anns]
    prompt = ' '.join(captions_list)
    try:
      response = client.images.generate(
        model="dall-e-2",
        prompt=prompt,
        size="512x512",
        n=1,
      )
    except Exception as e:
      print(f"An unexpected error occurred: {e}")
      continue
    image = response.data[0].url
    image = Image.open(requests.get(image, stream=True).raw)
    image.save(img_path)